## Corrections to TDX netCDF files

This code was necessary because the initial processing runs (with Multilooks of 4,8,16 and 32) had a bug, meaning that they didn't take into account which satellite was the primary satellite. 

For the ascending images, the height values from 11th Jan 2020 and 13th Feb 2020 had an extra negative sign. In the descending passes, the height values from 22nd Dec had an extra negative sign.

In addition, the ascending image from the 6th November, with too small a height of ambiguity, is dropped from the data.

In [1]:
import xarray as xr
import os 
import glob
from matplotlib import pyplot as plt
import numpy as np
import rioxarray as rx
import geopandas as gpd
from datetime import datetime as dt
import pandas as pd
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import ImageGrid

def open_ds(filename):
    ds = xr.open_dataset(filename).drop(['metadata','crs']).rio.write_crs('EPSG:4326')
    #if 'GAB' in filename:
    #    ds = ds.isel(t=slice(1,None)) # First image is waay off for some reason in Gabon
    return ds

data_dir = '/disk/scratch/local.4/harry/DEM/NC/merged/full/'
os.chdir(data_dir)
files=glob.glob('*_GAB_*.nc')
files

['TDX_DEM_GAB_SM_A_ML6_merged.nc',
 'TDX_DEM_GAB_SM_A_ML3_merged.nc',
 'TDX_DEM_GAB_SM_D_ML28_merged.nc',
 'TDX_DEM_GAB_SM_A_ML10_merged.nc',
 'TDX_DEM_GAB_SM_A_ML28_merged.nc',
 'TDX_DEM_GAB_SM_A_ML4_merged.nc',
 'TDX_DEM_GAB_SM_A_ML20_merged.nc',
 'TDX_DEM_GAB_SM_A_ML9_merged.nc',
 'TDX_DEM_GAB_SM_D_ML4_merged.nc',
 'TDX_DEM_GAB_SM_A_ML32_merged.nc',
 'TDX_DEM_GAB_SM_A_ML14_merged.nc',
 'TDX_DEM_GAB_SM_A_ML2_merged.nc',
 'TDX_DEM_GAB_SM_D_ML12_merged.nc',
 'TDX_DEM_GAB_SM_D_ML20_merged.nc',
 'TDX_DEM_GAB_SM_D_ML16_merged.nc',
 'TDX_DEM_GAB_SM_A_ML12_merged.nc',
 'TDX_DEM_GAB_SM_D_ML9_merged.nc',
 'TDX_DEM_GAB_SM_D_ML6_merged.nc',
 'TDX_DEM_GAB_SM_A_ML5_merged.nc',
 'TDX_DEM_GAB_SM_D_ML32_merged.nc',
 'TDX_DEM_GAB_SM_D_ML3_merged.nc',
 'TDX_DEM_GAB_SM_A_ML8_merged.nc',
 'TDX_DEM_GAB_SM_D_ML7_merged.nc',
 'TDX_DEM_GAB_SM_D_ML24_merged.nc',
 'TDX_DEM_GAB_SM_A_ML7_merged.nc',
 'TDX_DEM_GAB_SM_A_ML24_merged.nc',
 'TDX_DEM_GAB_SM_D_ML10_merged.nc',
 'TDX_DEM_GAB_SM_D_ML8_merged.nc',
 'TDX_

In [3]:
for f in files:
    ds= xr.open_dataset(f)
    if ds.t.size == 5:
        ds = ds.isel(t=slice(1,None))
    if '_ML4_' in f or '_ML8_' in f or '_ML16_' in f or '_ML32_' in f:
        if '_D_' in f:
            ds['height'][1,:,:] = -ds['height'][1,:,:]
        if '_A_' in f:
            ds['height'][0,:,:] = -ds['height'][0,:,:]
            ds['height'][-1,:,:] = -ds['height'][-1,:,:]
    out = f.split('.nc')[0] +'_corrected.nc'
    out = '/disk/scratch/local.4/harry/DEM/NC/merged/full/corrected/'+out
    ds.to_netcdf(out)